# Análisis de series financieras y volatilidad

Este cuaderno analiza el comportamiento de una acción bursátil (Apple Inc.) utilizando datos reales descargados desde Yahoo Finance.  
El objetivo es conectar los conceptos teóricos de series financieras con la práctica aplicada en Python.  
Cada paso incluye su explicación e intuición económica.

In [ ]:
# 1. Importación de librerías
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

plt.style.use('seaborn-v0_8-darkgrid')

print('Librerías cargadas correctamente.')

### Intuición
Las librerías conforman el entorno básico para análisis financiero en Python.  
- **pandas** organiza los datos.  
- **numpy** realiza cálculos numéricos.  
- **yfinance** descarga información real del mercado.  
- **matplotlib** y **seaborn** permiten visualizar los resultados.

In [ ]:
# 2. Descarga de datos de Apple (AAPL)
data = yf.download('AAPL', period='6mo', interval='1d')
data.head()

### Intuición
Cada fila representa un día de negociación.  
Las columnas muestran precios de apertura, máximo, mínimo, cierre y volumen.  
Estos datos son la base del análisis de rendimiento, riesgo y volatilidad.

In [ ]:
# 3. Limpieza y exploración
data.info()
data = data.rename(columns={'Open':'Apertura','High':'Maximo','Low':'Minimo','Close':'Cierre','Volume':'Volumen'})
data.head()

### Intuición
Siempre es necesario revisar la estructura de los datos.  
Aquí verificamos tipos de variables y renombramos columnas para mayor claridad.  
Esto previene errores en los cálculos posteriores.

In [ ]:
# 4. Cálculo de rendimientos diarios
data['Rendimiento'] = data['Cierre'].pct_change() * 100
data = data.dropna()
data['Rendimiento'].describe()

### Intuición
El rendimiento mide el cambio porcentual diario del precio.  
Es la variable principal en el análisis de series financieras.  
Permite cuantificar la rentabilidad y la volatilidad de un activo.

In [ ]:
# 5. Visualización de precios y rendimientos
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
axes[0].plot(data.index, data['Cierre'], color='blue')
axes[0].set_title('Evolución del precio de cierre de Apple (AAPL)')
axes[0].set_ylabel('Precio (USD)')

axes[1].plot(data.index, data['Rendimiento'], color='orange')
axes[1].set_title('Rendimientos diarios (%)')
axes[1].set_ylabel('% diario')

plt.tight_layout()
plt.show()

### Intuición
Los precios muestran la tendencia del activo, mientras los rendimientos reflejan su variabilidad.  
Las caídas o picos abruptos son evidencia de alta volatilidad.

In [ ]:
# 6. Medición de volatilidad
vol_diaria = data['Rendimiento'].std()
vol_anual = vol_diaria * np.sqrt(252)

print(f'Volatilidad diaria promedio: {vol_diaria:.2f}%')
print(f'Volatilidad anualizada: {vol_anual:.2f}%')

### Intuición
La desviación estándar de los rendimientos representa el riesgo del activo.  
Cuanto más alta, mayor incertidumbre en los retornos.  
La anualización permite comparar activos en horizontes temporales distintos.

In [ ]:
# 7. Distribución de rendimientos
sns.histplot(data['Rendimiento'], bins=40, kde=True, color='purple')
plt.title('Distribución de rendimientos diarios de Apple (AAPL)')
plt.xlabel('Rendimiento diario (%)')
plt.ylabel('Frecuencia')
plt.show()

### Intuición
La forma de la distribución revela si los rendimientos son normales o si presentan colas largas.  
Las colas pesadas implican una mayor probabilidad de eventos extremos, lo cual es común en los mercados financieros.

In [ ]:
# 8. Correlación entre varias acciones
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN']
multi = yf.download(tickers, period='3mo')['Close']
rend = multi.pct_change().dropna() * 100

sns.heatmap(rend.corr(), annot=True, cmap='coolwarm')
plt.title('Correlación entre rendimientos de acciones tecnológicas')
plt.show()

### Intuición
La correlación mide qué tan coordinadamente se mueven los activos.  
Correlaciones altas reducen los beneficios de diversificación.  
Por eso, es un concepto esencial en gestión de portafolios.

In [ ]:
# 9. Valor en Riesgo (VaR) empírico
var_95 = np.percentile(data['Rendimiento'], 5)
print(f'Valor en riesgo diario (95% confianza): {var_95:.2f}%')

### Intuición
El Valor en Riesgo indica la pérdida máxima esperada con un 95% de confianza.  
Si el VaR es -3%, significa que solo 5 de cada 100 días la pérdida superará el 3%.

In [ ]:
# 10. Conclusión
media = data['Rendimiento'].mean()
print('--- Resumen del análisis ---')
print(f'Rendimiento promedio diario: {media:.2f}%')
print(f'Volatilidad diaria: {vol_diaria:.2f}%')
print(f'Valor en riesgo (95%): {var_95:.2f}%')
print('\nInterpretación económica:')
print('La acción muestra una rentabilidad promedio positiva con riesgo moderado.')
print('El valor en riesgo refleja la magnitud de pérdidas esperadas en escenarios adversos.')